1. **Классификация текстовых отрывков**
   - Подбор архитектур RNN (LSTM, GRU) и их гиперпараметров для анализа естественных текстов (Чехов, Достоевский, Маяковский и пр.).
   - Оценка качества (Accuracy, Loss) обучения на различных наборах текстов и конфигурациях сетей.

In [ ]:
!pip install sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential, layers, optimizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score


In [ ]:

# 1. Settings
authors = ["Чехов", "Маяковский", "Достоевский", "Корь", "Чесотка", "Шизофрения"]
model_types = ["LSTM", "GRU"]
seq_length = 40
vocab_size = 4000      # adjust as needed
embedding_dim = 64
rnn_units = 64
batch_size = 32
epochs = 10            # for demo; increase as needed
dropout = 0.2
recurennt_dropout = 0.2
func_activation = "softmax" # defualt softmax
bidirectional_rnn_units = int(embedding_dim)

results = []


In [ ]:
# 2. Helper: build model
import tensorflow as tf
from tensorflow.keras import Model, Input, layers, optimizers

def build_model_with_self_attention(model_type):
    # 1) вход
    inp = Input(shape=(seq_length,), name="input_tokens")

    # 2) эмбеддинги
    x = layers.Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=seq_length,
        name="embedding"
    )(inp)

    # 3) Bi-RNN, возвращает всю последовательность
    if model_type == "LSTM":
        rnn = layers.LSTM(
            bidirectional_rnn_units,
            return_sequences=True,
            recurrent_dropout=recurennt_dropout,
            name="bi_lstm"
        )
    else:
        rnn = layers.GRU(
            bidirectional_rnn_units,
            return_sequences=True,
            recurrent_dropout=recurennt_dropout,
            name="bi_gru"
        )
    x = layers.Bidirectional(rnn, merge_mode="concat", name="bidirectional")(x)
    x = layers.Dropout(dropout, name="dropout_after_bi_rnn")(x)

    # 4) Self-Attention
    #    query = key = value = все тайм-степы из Bi-RNN
    attn = layers.MultiHeadAttention(
        num_heads=4,
        key_dim=bidirectional_rnn_units,    # обычно = units
        name="self_attention"
    )
    # в Functional API: attn(query, value)
    attn_out = attn(x, x)
    # добавим residual-связь
    x = layers.Add(name="residual_after_attention")([x, attn_out])

    # 5) Второй RNN без return_sequences
    if model_type == "LSTM":
        x = layers.LSTM(
            rnn_units,
            recurrent_dropout=recurennt_dropout,
            name="final_lstm"
        )(x)
    else:
        x = layers.GRU(
            rnn_units,
            recurrent_dropout=recurennt_dropout,
            name="final_gru"
        )(x)
    x = layers.Dropout(dropout, name="dropout_before_dense")(x)

    # 6) Выходной плотный слой
    out = layers.Dense(
        vocab_size,
        activation=func_activation,
        name="softmax_output"
    )(x)

    model = Model(inputs=inp, outputs=out, name=f"{model_type}_with_self_attention")

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=optimizers.Adam(3e-4, clipnorm=1.0),
        metrics=["accuracy"]
    )
    return model

# пример создания
model = build_model_with_self_attention("GRU")
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "GRU_with_self_attention"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_tokens        │ (None, 40)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 40, 64)    │    256,000 │ input_tokens[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 40, 128)   │     49,920 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_after_bi_r… │ (None, 40, 128)   │          0 │ bidirectional[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ self_attention      │ (None, 40, 128)   │    131,968 │ dropout_after_bi… │
│ (MultiHeadAttentio… │                   │            │ dropout_after_bi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ residual_after_att… │ (None, 40, 128)   │          0 │ dropout_after_bi… │
│ (Add)               │                   │            │ self_attention[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_gru (GRU)     │ (None, 64)        │     37,248 │ residual_after_a… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_before_den… │ (None, 64)        │          0 │ final_gru[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax_output      │ (None, 4000)      │    260,000 │ dropout_before_d… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 735,136 (2.80 MB)

 Trainable params: 735,136 (2.80 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 3. Loop over authors & model types
for author in authors:
    # 3.1 Load & preprocess text
    with open(f"/content/drive/MyDrive/Text_MLmedicine/data/{author}.txt", encoding="utf-8") as f:
        text = f.read().lower()
    # simple clean: keep cyrillic, whitespace, basic punctuation
    text = tf.strings.regex_replace(text, r"[^а-яё0-9\s\.,;:!\?—\-]", " ")
    text = tf.strings.regex_replace(text, r"\s+", " ").numpy().decode().strip()

    # 3.2 Tokenize with Keras Tokenizer
    tok = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token="<unk>")
    tok.fit_on_texts([text])
    seq = tok.texts_to_sequences([text])[0]

    # 3.3 Create input-output pairs
    X, y = [], []
    for i in range(len(seq) - seq_length):
        X.append(seq[i:i+seq_length])
        y.append(seq[i+seq_length])
    X = np.array(X)
    y = np.array(y)

    # 3.4 Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=5641)

    for mtype in model_types:
        print(f"Training {mtype} for {author}...")
        model = build_model_with_self_attention(mtype)
        # callback to save best
        cb = callbacks.EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
        history = model.fit(
            X_train, y_train,
            validation_split=0.2,
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[cb],
            verbose=1
        )

        # 3.5 Evaluate
        loss, acc = model.evaluate(X_test, y_test, verbose=0)
        # predict probabilities
        y_pred_probs = model.predict(X_test, verbose=0)
        y_pred = np.argmax(y_pred_probs, axis=1)
        # metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        ev = explained_variance_score(y_test, y_pred)

        # 3.6 Record
        results.append({
            "Author": author,
            "Model": mtype,
            "Embedding dim": embedding_dim,
            "RNN units": rnn_units,
            "Dropout": dropout,
            "Epochs": len(history.history["loss"]),
            "Batch size": batch_size,
            "Final Loss": round(loss, 4),
            "Final Accuracy": round(acc, 4),
            "MSE": round(mse, 4),
            "RMSE": round(rmse, 4),
            "MAE": round(mae, 4),
            "R2": round(r2, 4),
            "Explained Variance": ev
        })



Training LSTM for Чехов...
Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 73s 257ms/step - accuracy: 0.0359 - loss: 7.9377 - val_accuracy: 0.0490 - val_loss: 7.2815
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 80s 250ms/step - accuracy: 0.0528 - loss: 6.9724 - val_accuracy: 0.0490 - val_loss: 7.5235
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 79s 235ms/step - accuracy: 0.0516 - loss: 6.8879 - val_accuracy: 0.0490 - val_loss: 7.6462
Training GRU for Чехов...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


222/222 ━━━━━━━━━━━━━━━━━━━━ 58s 210ms/step - accuracy: 0.0439 - loss: 7.9430 - val_accuracy: 0.0490 - val_loss: 7.2699
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 80s 200ms/step - accuracy: 0.0540 - loss: 6.9288 - val_accuracy: 0.0490 - val_loss: 7.4647
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 44s 198ms/step - accuracy: 0.0489 - loss: 6.8450 - val_accuracy: 0.0490 - val_loss: 7.6198
Training LSTM for Маяковский...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 21s 349ms/step - accuracy: 0.0241 - loss: 8.2929 - val_accuracy: 0.0114 - val_loss: 8.2867
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 200ms/step - accuracy: 0.0564 - loss: 8.2429 - val_accuracy: 0.0114 - val_loss: 8.1607
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 295ms/step - accuracy: 0.0436 - loss: 7.7030 - val_accuracy: 0.0114 - val_loss: 7.9909
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - accuracy: 0.0371 - loss: 7.0221 - val_accuracy: 0.0114 - val_loss: 8.0098
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 283ms/step - accuracy: 0.0400 - loss: 6.6234 - val_accuracy: 0.0114 - val_loss: 8.0901
Training GRU for Маяковский...
Epoch 1/10


...

Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 21s 276ms/step - accuracy: 0.0101 - loss: 8.2931 - val_accuracy: 0.0282 - val_loss: 8.2871
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 7s 277ms/step - accuracy: 0.0485 - loss: 8.2472 - val_accuracy: 0.0282 - val_loss: 8.0261
Epoch 3/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 6s 214ms/step - accuracy: 0.0416 - loss: 7.4861 - val_accuracy: 0.0282 - val_loss: 7.6635
Epoch 4/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 12s 262ms/step - accuracy: 0.0387 - loss: 6.7937 - val_accuracy: 0.0282 - val_loss: 7.5888
Epoch 5/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 11s 287ms/step - accuracy: 0.0383 - loss: 6.4480 - val_accuracy: 0.0282 - val_loss: 7.6110
Epoch 6/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 6s 215ms/step - accuracy: 0.0437 - loss: 6.2520 - val_accuracy: 0.0282 - val_loss: 7.6779
Training LSTM for Шизофрения...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 17s 265ms/step - accuracy: 0.0046 - loss: 8.2934 - val_accuracy: 0.0333 - val_loss: 8.2899
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 285ms/step - accuracy: 0.0220 - loss: 8.2740 - val_accuracy: 0.0333 - val_loss: 8.2173
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 191ms/step - accuracy: 0.0418 - loss: 8.0098 - val_accuracy: 0.0333 - val_loss: 7.9303
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 233ms/step - accuracy: 0.0472 - loss: 7.2796 - val_accuracy: 0.0333 - val_loss: 7.7492
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 186ms/step - accuracy: 0.0296 - loss: 6.7986 - val_accuracy: 0.0333 - val_loss: 7.7009
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 262ms/step - accuracy: 0.0317 - loss: 6.5112 - val_accuracy: 0.0333 - val_loss: 7.7185
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 186ms/step - accuracy: 0.0392 - loss: 6.3145 - val_accuracy: 0.0333 - val_loss: 7.7639
Training GRU for Шизофрения...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 365ms/step - accuracy: 0.0156 - loss: 8.2934 - val_accuracy: 0.0333 - val_loss: 8.2894
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 7s 194ms/step - accuracy: 0.0449 - loss: 8.2762 - val_accuracy: 0.0333 - val_loss: 8.2418
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 285ms/step - accuracy: 0.0342 - loss: 8.0320 - val_accuracy: 0.0333 - val_loss: 7.8719
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 195ms/step - accuracy: 0.0295 - loss: 7.1654 - val_accuracy: 0.0333 - val_loss: 7.7259
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 7s 266ms/step - accuracy: 0.0371 - loss: 6.7222 - val_accuracy: 0.0333 - val_loss: 7.6930
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 200ms/step - accuracy: 0.0446 - loss: 6.4265 - val_accuracy: 0.0333 - val_loss: 7.7208
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 260ms/step - accuracy: 0.0348 - loss: 6.2756 - val_accuracy: 0.0333 - val_loss: 7.7764


In [ ]:
# 4. Save results table
df = pd.DataFrame(results)
df.to_csv("model_comparison_results_self_attention_more.csv", index=False)
print("Done. Results saved to model_comparison_results.csv")

Done. Results saved to model_comparison_results.csv


In [ ]:
df

,Author,Model,Embedding dim,RNN units,Dropout,Epochs,Batch size,Final Loss,Final Accuracy,MSE,RMSE,MAE,R2,Explained Variance
0,Чехов,LSTM,64,64,0.2,3,32,7.3243,0.0433,1.885206e+06,1373.0280,805.6142,-0.5250,0.0
1,Чехов,GRU,64,64,0.2,3,32,7.3360,0.0433,1.885206e+06,1373.0280,805.6142,-0.5250,0.0
2,Маяковский,LSTM,64,64,0.2,5,32,7.7807,0.0594,1.278292e+05,357.5321,254.1050,-1.0160,0.0
3,Маяковский,GRU,64,64,0.2,5,32,7.7863,0.0594,1.278292e+05,357.5321,254.1050,-1.0160,0.0
4,Достоевский,LSTM,64,64,0.2,3,32,7.1098,0.0638,1.619774e+06,1272.7035,696.9266,-0.4282,0.0
5,Достоевский,GRU,64,64,0.2,3,32,7.1006,0.0638,1.619774e+06,1272.7035,696.9266,-0.4282,0.0
6,Корь,LSTM,64,64,0.2,5,32,7.2298,0.0243,1.073049e+05,327.5743,231.2693,-0.9929,0.0
7,Корь,GRU,64,64,0.2,5,32,7.2302,0.0243,1.073049e+05,327.5743,231.2693,-0.9929,0.0
8,Чесотка,LSTM,64,64,0.2,6,32,7.4548,0.0226,1.011757e+05,318.0812,221.1541,-0.9348,0.0
9,Чесотка,GRU,64,64,0.2,6,32,7.4660,0.0301,1.016188e+05,318.7771,222.0940,-0.9433,0.0


In [ ]:
# 4) Функция генерации
UNK_ID = sp.PieceToId("<unk>")  # обычно стоит 0, но на всякий случай

def generate_text(model, sp, seed_text, gen_length=200, temperature=1.0):
    # 4.1) Токенизируем seed
    seed_ids = sp.EncodeAsIds(seed_text)
    # если seed короче чем seq_length — дополняем слева нулями
    if len(seed_ids) < seq_length:
        seed_ids = [0] * (seq_length - len(seed_ids)) + seed_ids
    else:
        seed_ids = seed_ids[-seq_length:]

    generated = seed_ids.copy()

    for _ in range(gen_length):
        # готовим батч размером (1, seq_length)
        x_pred = np.array(generated[-seq_length:])[None, :]
        # предсказываем следующий логит
        preds = model.predict(x_pred, verbose=0)[0]   # shape=(vocab_size,)
        # сэмплируем id
        next_id = sample_with_temperature(preds, temperature)
        # next_id = top_k_sample(preds, 5)

        # если это <unk> — пропускаем и пробуем ещё раз
        if next_id == UNK_ID:
            continue

        generated.append(next_id)

    # декодируем весь список обратно в строку
    return sp.DecodeIds(generated)



In [ ]:
import re

prompt = "Поручик Дубов уже не молодой армейский служака"
generated = generate_text(model, sp, prompt, gen_length=50, temperature=0.8)

# допустимый набор: кириллица, латиница, знаки пунктуации, пробел
pattern = re.compile(r"[^a-яА-ЯёЁa-zA-Z0-9\.\,\!\?\:\;\—\«\»\s]")
cleaned = pattern.sub("", generated)
print( cleaned )

                                                         оручик   убов уже не молодой армейский служака,, и где которую он не что и эту у — глаза, на радости я что и может то ложитесь. сонные вот озабоченно? лицо никитин, как тени и угрызения ее, себя,, на. человек никакой из, барыня в! что,
